In [1]:
import json
import math
import os, sys, shutil
import numpy as np
from matplotlib import pyplot as plt

In [2]:
class Point(object):
    def __init__(self, x, y, c):
        self.x = x     # x_coordinate of the point
        self.y = y     # y_coordinate of the point
        self.c = c     # confidence of the point

    def __repr__(self):
        return 'x: '+ str(self.x) + ", y: " +str(self.y) + ", c: " + str(self.c)

### Get input video json directories
Append directories to list

In [3]:
base_dir = '/Users/berkeesmer/Desktop/titanik_data/MVI_8191_JSON/json_664_697/'
json_dir = sorted(['{}/{}'.format(base_dir, file_name) for file_name in os.listdir(base_dir)])

### Read Json Files

In [4]:
json_data_array = []

# read jsons
for item_index in range(len(json_dir)):
    json_path = json_dir[item_index]
    with open(json_path) as json_file:
        json_data = json.loads(json_file.read())
        json_data_array.append(json_data)

### Extract Features
<br>
<br>
<div class="row" style="content: "";
  clear: both;
  display: table;">
  <div class="column">
        <img src="https://github.com/CMU-Perceptual-Computing-Lab/openpose/raw/master/doc/media/keypoints_pose_25.png" style="float: left;
  width: 30%;
  padding: 5px;">
    </div>
    <div class="column">
        <img src="https://github.com/CMU-Perceptual-Computing-Lab/openpose/raw/master/doc/media/keypoints_hand.png" style="float: right;
  width: 40%;
  padding: 5px;">
    </div>
</div>

<img src="https://github.com/CMU-Perceptual-Computing-Lab/openpose/raw/master/doc/media/keypoints_face.png" style="
  width: 50%;">

In [5]:
def isFingerActive(p1, p2, p3):
    # p2 is the cross point
    len_p1_p2 = math.sqrt(math.pow(p1.x-p2.x, 2) + math.pow(p1.y-p2.y, 2))
    len_p1_p3 = math.sqrt(math.pow(p1.x-p3.x, 2) + math.pow(p1.y-p3.y, 2))
    len_p2_p3 = math.sqrt(math.pow(p2.x-p3.x, 2) + math.pow(p2.y-p3.y, 2))
        
    try:
        angle_in_radians = math.acos((math.pow(len_p1_p2, 2) + math.pow(len_p2_p3, 2) - math.pow(len_p1_p3, 2)) / (2 * len_p1_p2 * len_p2_p3))
        angle_in_degrees = math.degrees(angle_in_radians)
    
        return 1 if angle_in_degrees > 120 else 0
    
    except ValueError:
        return 1
    
    except ZeroDivisionError:
        return 1

In [6]:
def isFingerActive2(p, c):
    len_p_c = math.sqrt(math.pow(p.x-c.x, 2) + math.pow(p.y-c.y, 2))
    return 1 if len_p_c > 45 else 0

In [7]:
result_arrs = []

for json_index in range(len(json_data_array)):
    person = json_data_array[json_index]['people'][0]
    left_hand = person['hand_left_keypoints_2d']
    right_hand = person['hand_right_keypoints_2d']

    ########## EXTRACT LEFT HAND POINTS  ##########
    i = 0
    lhand_points = []
    for i in range(0, len(left_hand), 3):
        point = Point(left_hand[i], left_hand[i+1], left_hand[i+2])
        lhand_points.append(point)
        
    ########## EXTRACT RIGHT HAND POINTS  ##########
    i = 0
    rhand_points = []
    for i in range(0, len(right_hand), 3):
        point = Point(right_hand[i], right_hand[i+1], right_hand[i+2])
        rhand_points.append(point)

    confidence_threshold = 0.01

    # CALCULATE LEFT FINGERS
    l_thumb_angle = isFingerActive2(lhand_points[4], lhand_points[0])
    l_index_angle = isFingerActive2(lhand_points[8], lhand_points[0])
    l_middle_angle = isFingerActive2(lhand_points[12], lhand_points[0])
    l_ring_angle = isFingerActive2(lhand_points[16], lhand_points[0])
    l_pinky_angle = isFingerActive2(lhand_points[20], lhand_points[0])

    """ Left Thresholding
    if lhand_points[4].c < threshold:
        l_thumb = 0

    if lhand_points[8].c < threshold:
        l_index = 0

    if lhand_points[12].c < threshold:
        l_middle = 0

    if lhand_points[16].c < threshold:
        l_ring = 0

    if lhand_points[20].c < threshold:
        l_pinky = 0
    """

    # CALCULATE RIGHT FINGERS
    r_thumb_angle = isFingerActive2(rhand_points[4], rhand_points[0])
    r_index_angle = isFingerActive2(rhand_points[8], rhand_points[0])
    r_middle_angle = isFingerActive2(rhand_points[12], rhand_points[0])
    r_ring_angle = isFingerActive2(rhand_points[16], rhand_points[0])
    r_pinky_angle = isFingerActive2(rhand_points[20], rhand_points[0])
    
    """ Right Thresholding
    if rhand_points[4].c < threshold:
        r_thumb = 0

    if rhand_points[8].c < threshold:
        r_index = 0

    if rhand_points[12].c < threshold:
        r_middle = 0

    if rhand_points[16].c < threshold:
        r_ring = 0

    if rhand_points[20].c < threshold:
        r_pinky = 0
    """
    
    json_left_angles = [l_thumb_angle, l_index_angle, l_middle_angle, l_ring_angle, l_pinky_angle]
    json_right_angles = [r_thumb_angle, r_index_angle, r_middle_angle, r_ring_angle, r_pinky_angle]    
    result_arrs.append((json_left_angles.count(1), json_right_angles.count(1)))

In [8]:
start_index = 664

for i in range(len(json_data_array)):
    print("Frame {}: {}".format(start_index+i, result_arrs[i]))

Frame 664: (5, 5)
Frame 665: (5, 4)
Frame 666: (5, 5)
Frame 667: (5, 5)
Frame 668: (5, 5)
Frame 669: (5, 5)
Frame 670: (5, 5)
Frame 671: (5, 5)
Frame 672: (5, 3)
Frame 673: (5, 2)
Frame 674: (5, 2)
Frame 675: (5, 3)
Frame 676: (4, 3)
Frame 677: (5, 3)
Frame 678: (4, 3)
Frame 679: (4, 3)
Frame 680: (5, 1)
Frame 681: (5, 5)
Frame 682: (5, 5)
Frame 683: (5, 5)
Frame 684: (5, 5)
Frame 685: (5, 5)
Frame 686: (3, 5)
Frame 687: (2, 5)
Frame 688: (5, 5)
Frame 689: (5, 5)
Frame 690: (5, 5)
Frame 691: (5, 5)
Frame 692: (5, 5)
Frame 693: (4, 5)
Frame 694: (5, 5)
Frame 695: (5, 5)
Frame 696: (5, 5)
Frame 697: (5, 5)
